<a href="https://colab.research.google.com/github/Maksim-Fedosenko/ML_course_DPO/blob/main/full_true_preprocesssing_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

+

In [ ]:
import pandas as pd
import datetime as dt
import dask.dataframe as dd

"+" но чёто показатели разные

In [ ]:
df_data_hist=pd.read_csv('hist_trx.csv')

df_data_hist.shape

(562490, 11)

+

In [ ]:
df_data_hist=pd.read_csv('hist_trx.csv')

In [ ]:
df_data_hist.head

<bound method _Frame.head of Dask DataFrame Structure:
              Unnamed: 0 event_id date_time user_id sub_channel event_type sub_type atm_mcc mcc_group atm_merchant_name   amount
npartitions=1                                                                                                                   
                   int64   object    object   int64      object     object   object   int64    object            object  float64
                     ...      ...       ...     ...         ...        ...      ...     ...       ...               ...      ...
Dask Name: read-csv, 1 tasks>

In [ ]:
df_data_trx=pd.read_csv('hits0712.csv', index_col=0)
df_data_trx.shape

(9016, 6)

In [ ]:
df_data_trx.head(5)


,event_id,user_id,rec_user_id,date_time,resolution,sub_channel
5,0fcea7fec4ff479cac8cf37b4555c817,75301207,49913000,2018-12-07 07:56:49,G,ATMAPI
6,c29a4e64d27b435b9b55aa3e62ce54d4,1200695,12797310,2018-12-07 07:14:41,G,MOBILEAPI
8,3107b2b1afcb490ab0a31135eb4b386c,45657780,23814017,2018-12-07 14:02:30,G,MOBILEAPI
10,d9bc95d991144d53b950d8084fa846bf,21683486,9865131,2018-12-07 10:32:41,G,WEBAPI
22,9fc959da4cd144ea8bf5bae2dc041a5e,4531894,VSP15472255,2018-12-07 18:22:52,G,MOBILEAPI


In [ ]:
df_data_trx.dtypes

event_id       object
user_id        object
rec_user_id    object
date_time      object
resolution     object
sub_channel    object
dtype: object

In [ ]:
df_data_trx.describe()

,event_id,user_id,rec_user_id,date_time,resolution,sub_channel
count,9016,9016,9016,9016,9016,9016
unique,9016,8507,7257,8372,3,3
top,0fcea7fec4ff479cac8cf37b4555c817,20646287,71370973,2018-12-07 10:36:34,G,MOBILEAPI
freq,1,8,340,3,8943,6528


In [ ]:
df_data_trx.resolution.value_counts()

G    8943
F      63
S      10
Name: resolution, dtype: int64

In [ ]:
df_data_trx[['rec_user_id','resolution']].drop_duplicates().head()

,rec_user_id,resolution
5,49913000,G
6,12797310,G
8,23814017,G
10,9865131,G
22,VSP15472255,G


In [ ]:
target = df_data_trx[['rec_user_id','resolution']].drop_duplicates()

In [ ]:
resolution_map = {'G': 0, 'F': 1, 'S': 1}
target['class'] = target.resolution.map(lambda x: resolution_map[x])

Raw trans data preprocessing

In [ ]:
df_data_hist.head()

,Unnamed: 0,event_id,date_time,user_id,sub_channel,event_type,sub_type,atm_mcc,mcc_group,atm_merchant_name,amount
0,0,22c9f1ac686a43e18cdb798489193238,2018-12-06 09:33:21,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5921.0,R,YUG 426,280.0
1,1,62dac13fac68416d9bc340c51ddcb977,2018-12-06 09:36:08,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5331.0,R,MAGAZIN RODINA,376.9
2,2,9b666b8c9d4d4faea78e2b28a5468794,2018-12-06 12:21:02,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5921.0,R,YUG 426,143.0
3,3,d4d805fc3d5f4c91aa9b0389333b780c,2018-12-06 07:46:58,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5331.0,R,MAGAZIN RODINA,162.0
4,4,1d162351117a4418949dcf94111b9964,2018-12-06 12:21:27,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5331.0,R,MAGAZIN RODINA,56.0


In [ ]:
datahist_clean = df_data_hist[df_data_hist.date_time < '2018-12-07']

In [ ]:
df_data_hist.shape[0]

562490

In [ ]:
datahist_cleanest = dd.merge(datahist_clean, target, left_on= 'user_id', right_on= 'rec_user_id', how= 'inner')

In [ ]:
datahist_cleanest.columns

Index(['Unnamed: 0', 'event_id', 'date_time', 'user_id', 'sub_channel',
       'event_type', 'sub_type', 'atm_mcc', 'mcc_group', 'atm_merchant_name',
       'amount', 'rec_user_id', 'resolution', 'class'],
      dtype='object')

In [ ]:
datahist_cleanest.shape[0]

269701

In [ ]:
datahist_cleanest[['user_id','mcc_group']].to_csv('history_cleaned-*.csv', header=False)

In [ ]:
!head -n 5 history_cleaned-1.csv

head: cannot open 'history_cleaned-1.csv' for reading: No such file or directory


In [ ]:
from glob import glob
filenames = glob('history_cleaned-*.csv')
with open('history_cleaned.csv','w') as out:
  for fn in filenames:
    with open(fn) as f:
      out.write(f.read())

In [ ]:
all_data = pd.read_csv('history_cleaned.csv', header=0, names=['index', 'rec_user_id', 'mcc_group'])

In [ ]:
all_data.head()

,index,rec_user_id,mcc_group
0,1,102050167,R
1,2,102050167,R
2,3,102050167,R
3,4,102050167,R
4,5,102050167,R


In [ ]:
all_data.mcc_group.value_counts()

R    86474
C    82975
U    52605
Z    31511
F    10366
X     2546
T     1182
A      660
H      354
J      238
Q      105
O       31
Name: mcc_group, dtype: int64

In [ ]:
oec_data = pd.pivot_table(all_data, index='rec_user_id',columns=['mcc_group'], values='index', aggfunc='count', fill_value=0)

In [ ]:
target_all = pd.merge(oec_data, target[['rec_user_id', 'class']], on='rec_user_id')

In [ ]:
target_all.head()

,rec_user_id,A,C,F,H,J,O,Q,R,T,U,X,Z,class
0,100017681,0,5,0,0,0,0,0,14,0,9,1,42,0
1,100023450,0,23,0,0,0,0,0,31,0,19,0,2,0
2,100031471,0,1,0,0,0,0,0,6,0,0,0,0,0
3,10008487,0,24,6,0,0,0,0,21,0,41,1,7,0
4,10009395,0,6,0,0,0,0,0,13,0,16,0,0,0


In [ ]:
target_all[target_all['class']!=0]

,rec_user_id,A,C,F,H,J,O,Q,R,T,U,X,Z,class
19,10029019,0,2,1,0,0,0,0,13,2,0,0,6,1
91,101272872,0,3,5,0,0,0,0,30,0,1,0,11,1
645,106157134,0,6,0,0,0,0,0,10,0,5,0,0,1
714,106439794,0,92,0,0,0,0,0,6,0,2,0,116,1
868,106849367,0,10,0,0,0,0,0,0,0,0,0,8,1
1576,19134747,0,0,0,0,0,0,0,1,0,2,0,0,1
2036,26024565,0,33,0,0,0,0,0,9,0,24,0,2,1
2067,26599535,0,8,1,0,0,0,0,23,0,6,0,10,1
2581,34527832,0,0,1,0,0,0,0,12,0,2,0,0,1
2629,35184837,0,0,0,0,0,0,0,5,0,0,0,1,1


In [ ]:
target_all['class'].value_counts()

0    6802
1      47
Name: class, dtype: int64

In [ ]:
Xdata = target_all.drop(['class', 'rec_user_id'], axis=1)
ydata = target_all['class']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(Xdata, ydata, test_size=0.2, random_state=666)

In [ ]:
Xtest.shape

(1370, 12)

In [ ]:
ytrain.shape

(5479,)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(class_weight={0: 1, 1: 15})
clf.fit(Xtrain, ytrain)

RandomForestClassifier(class_weight={0: 1, 1: 15})

In [ ]:
ypred = pd.Series(clf.predict(Xtest), index=Xtest.index)

In [ ]:
ytest.value_counts()

0    1361
1       9
Name: class, dtype: int64

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
confusion_matrix(ytest, ypred)

array([[1357,    4],
       [   8,    1]])

In [ ]:
target_names = ['Genuine', 'Fraud']
print(classification_report(ytest, ypred, target_names=target_names))

              precision    recall  f1-score   support

     Genuine       0.99      1.00      1.00      1361
       Fraud       0.20      0.11      0.14         9

    accuracy                           0.99      1370
   macro avg       0.60      0.55      0.57      1370
weighted avg       0.99      0.99      0.99      1370



Проверка фрода

In [ ]:
ypred[ypred == 1].index.intersection(ytest[ytest == 1].index)

Int64Index([6547], dtype='int64')

In [ ]:
target_all.iloc[6547]

rec_user_id    MBK29495696
A                        0
C                        0
F                        0
H                        0
J                        0
O                        0
Q                        0
R                       54
T                        0
U                        0
X                        0
Z                       56
class                    1
Name: 6547, dtype: object

In [ ]:
df_data_hist[df_data_hist.user_id == '104779222'].head(100)

,Unnamed: 0,event_id,date_time,user_id,sub_channel,event_type,sub_type,atm_mcc,mcc_group,atm_merchant_name,amount
36482,36482,eb0a530096fd493cbffbc8bd796d1fc2,2018-12-06 09:51:35,104779222,ISSUER_ACQUIRER,PAYMENT,ATM_P2P_DEBIT,4829.0,U,SBOL,97001.0
36483,36483,038c54df1b0d4fe0a11899d8ac5b1a88,2018-12-06 14:31:00,104779222,ISSUER_ACQUIRER,PAYMENT,ATM_P2P_DEBIT,4829.0,U,SBOL,76500.0
36484,36484,500dbdc9e5524dc7b66c64f77f473131,2018-12-06 10:00:31,104779222,ISSUER_ACQUIRER,PAYMENT,ATM_P2P_DEBIT,4829.0,U,SBOL,96000.0
36485,36485,693b1de4aea440f585b5e73a727aa935,2018-12-06 10:00:09,104779222,ISSUER_ACQUIRER,PAYMENT,ATM_P2P_DEBIT,4829.0,U,SBOL,98000.0
36486,36486,ba4f4add78c84edda195fb8d0c99a1ba,2018-12-06 09:50:43,104779222,ISSUER_ACQUIRER,PAYMENT,ATM_P2P_DEBIT,4829.0,U,SBOL,97000.0
...,...,...,...,...,...,...,...,...,...,...,...
265319,24910,97dd2aa34b8a482f922d8952777d142e,2018-12-02 15:40:46,104779222,ISSUER,DEPOSIT,ATM_P2P_CREDIT,6536.0,R,VRB_MC,14200.0
265320,24911,d2ef8a1a7730433f938530bc1507fdbd,2018-12-02 15:40:44,104779222,ISSUER,DEPOSIT,ATM_P2P_CREDIT,6536.0,R,VRB_MC,14200.0
265321,24912,7dd1877f3763438ca5c1bde1a231296c,2018-12-02 15:40:40,104779222,ISSUER,DEPOSIT,ATM_P2P_CREDIT,6536.0,R,VRB_MC,14200.0
265322,24913,8b26d1f04dd846d6af6b194e934bc75b,2018-12-02 15:40:43,104779222,ISSUER,DEPOSIT,ATM_P2P_CREDIT,6536.0,R,VRB_MC,14200.0
